<a href="https://colab.research.google.com/github/RealllllXL/Python_project/blob/master/Ricetype_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install opendatasets --quiet
import opendatasets as od
dataset_url = 'https://www.kaggle.com/datasets/mssmartypants/rice-type-classification'
od.download(dataset_url)

Skipping, found downloaded files in "./rice-type-classification" (use force=True to force download)


In [3]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
data_df = pd.read_csv("/content/rice-type-classification/riceClassification.csv")
data_df.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [5]:
data_df.dropna(inplace=True) #drop any missing values
data_df.drop("id", axis=1, inplace=True) #delete id column,coz not necessary.axis=1 means colum
data_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [6]:
print(data_df["Class"].unique())

[1 0]


In [8]:
print(data_df["Class"].value_counts())

Class
1    9985
0    8200
Name: count, dtype: int64


In [9]:
# Normalization 每列元素除以最大的
original_df = data_df.copy()
for column in data_df.columns:
    data_df[column] = data_df[column] / data_df[column].abs().max()
data_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3,0.300979,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
4,0.361704,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


In [34]:
X = np.array(data_df.iloc[:, :-1]) #iloc:take specific part of data; :-1_All expect last one
Y = np.array(data_df.iloc[:, -1])


In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3) #30% test,70% train
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

In [36]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(12729, 10)
(2728, 10)
(2728, 10)


In [41]:
#convert dataset to pytorch dataset object
class dataset(Dataset):
  def __init__(self, X, Y):
     self.X = torch.tensor(X, dtype = torch.float32).to(device)
     self.Y = torch.tensor(Y, dtype = torch.float32).to(device)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index): #get one row
     return self.X[index], self.Y[index]

In [42]:
#create object
training_data = dataset(X_train, y_train)
validation_data = dataset(X_val, y_val)
testing_data = dataset(X_test, y_test)

In [43]:
#create dataloader
train_dataloader = DataLoader(training_data, batch_size=8, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=8, shuffle=True)
testing_dataloader = DataLoader(testing_data, batch_size=8, shuffle=True)

In [44]:
for x,y in train_dataloader:
  print(x)
  print("========")
  print(y)
  break

tensor([[0.7886, 0.7903, 0.8693, 0.8984, 0.7475, 0.8881, 0.8357, 0.6971, 0.8900,
         0.5158],
        [0.4713, 0.6489, 0.6354, 0.9283, 0.4503, 0.6865, 0.6552, 0.5581, 0.8298,
         0.5794],
        [0.5495, 0.7863, 0.6126, 0.9686, 0.5248, 0.7413, 0.7902, 0.6448, 0.7247,
         0.7283],
        [0.9167, 0.8823, 0.9074, 0.9166, 0.8761, 0.9575, 0.7124, 0.7722, 0.8431,
         0.5516],
        [0.3993, 0.7135, 0.4923, 0.9831, 0.3852, 0.6319, 0.6532, 0.5781, 0.6554,
         0.8222],
        [0.6248, 0.8843, 0.6170, 0.9819, 0.5971, 0.7904, 0.7253, 0.6998, 0.6998,
         0.8132],
        [0.7842, 0.8099, 0.8421, 0.9138, 0.7462, 0.8856, 0.6602, 0.7026, 0.8712,
         0.5457],
        [0.5182, 0.7689, 0.5940, 0.9697, 0.4924, 0.7199, 0.5558, 0.6241, 0.7298,
         0.7344]], device='cuda:0')
tensor([0., 1., 1., 0., 1., 1., 0., 1.], device='cuda:0')
